# <center> One Pixel Attack  
## <center> Tromper un modèle de classification d'images

### Introduction

L'attaque One Pixel (One Pixel Attack en anglais) est une technique d'attaque ciblée visant à tromper les modèles de classification d'images en modifiant uniquement quelques pixels de l'image d'entrée. Cette attaque démontre la vulnérabilité des modèles de deep learning face à des perturbations minimes et imperceptibles pour les humains, mais qui peuvent induire en erreur les algorithmes de classification.

Dans ce notebook, nous allons illustrer le fonctionnement de l'attaque One Pixel en utilisant un modèle de classification d'images pré-entraîné (ResNet50) et en effectuant une attaque sur une image d'exemple.

#### Étape 1 : Préparation de l'environnement

Commençons par importer les bibliothèques nécessaires et charger le modèle pré-entraîné (ResNet50) :



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy.optimize import differential_evolution

# Charger le modèle de classification d'images pré-entraîné (ResNet50)
model = keras.applications.ResNet50(weights='imagenet')


#### Étape 2 : Chargement de l'image

Nous allons charger l'image d'entrée sur laquelle nous allons effectuer l'attaque One Pixel. Assurez-vous d'avoir une image appropriée dans le répertoire et de fournir le chemin correct :

In [ ]:
# Chargement de l'image d'entrée
image = keras.preprocessing.image.load_img('input_image.jpg', target_size=(224, 224))
image = keras.preprocessing.image.img_to_array(image) / 255.0


#### Étape 3 : Définition de l'objectif de l'attaque

Choisissons la classe de sortie que nous souhaitons induire le modèle à prédire de manière incorrecte. Dans cet exemple, nous utiliserons la classe "chien" (indice 207) :

In [ ]:
# Classe ciblée pour l'attaque (ex. "chien" dans le cas d'un modèle ImageNet)
target_class = 207


#### Étape 4 : Implémentation de l'attaque One Pixel

Nous allons maintenant implémenter l'attaque One Pixel en utilisant l'algorithme d'optimisation différentielle (differential_evolution) pour trouver les coordonnées du pixel et les valeurs de couleur qui maximisent la probabilité de mauvaise classification pour la classe ciblée :

In [ ]:
# Fonction pour évaluer la probabilité de mauvaise classification
def evaluate_attack(x, image, target_class):
    perturbed_image = image.copy()
    perturbed_image[x[0], x[1], :] = x[2:]
    perturbed_image = np.expand_dims(perturbed_image, axis=0)
    predictions = model.predict(perturbed_image)
    predicted_class = np.argmax(predictions[0])
    return 1.0 - predictions[0][target_class]

# Fonction pour effectuer l'attaque One Pixel
def one_pixel_attack(image, target_class):
    bounds = [(0, image.shape[0]-1), (0, image.shape[1]-1), (0, 255), (0, 255), (0, 255)]
    result = differential_evolution(evaluate_attack, bounds, args=(image, target_class))
    best_pixel = result.x[:3].astype(int)
    best_color = result.x[3:].astype(int)
    return best_pixel, best_color

# Effectuer l'attaque One Pixel
best_pixel, best_color = one_pixel_attack(image, target_class)


#### Étape 5 : Évaluation de l'attaque

Une fois l'attaque One Pixel effectuée, nous allons évaluer le résultat en générant l'image perturbée et en utilisant le modèle pour prédire la classe de l'image modifiée :

In [ ]:
# Générer l'image perturbée
perturbed_image = image.copy()
perturbed_image[best_pixel[0], best_pixel[1], :] = best_color / 255.0

# Prédiction du modèle sur l'image perturbée
perturbed_image = np.expand_dims(perturbed_image, axis=0)
predictions = model.predict(perturbed_image)
predicted_class = np.argmax(predictions[0])
predicted_label = keras.applications.resnet50.decode_predictions(predictions, top=1)[0][0][1]


#### Résultats

Affichons maintenant les résultats de l'attaque, y compris les coordonnées et les valeurs du pixel modifié, ainsi que la classe prédite par le modèle pour l'image perturbée :

In [ ]:
print("Pixel modifié :", best_pixel)
print("Couleur modifiée :", best_color)
print("Classe prédite :", predicted_class)
print("Étiquette prédite :", predicted_label)

Ce notebook a illustré le processus de l'attaque One Pixel en utilisant un modèle de classification d'images pré-entraîné. Il est important de noter que cet exemple est fourni à des fins éducatives pour illustrer le concept de l'attaque One Pixel. Assurez-vous de respecter les lois et les règlements en vigueur concernant l'utilisation éthique et responsable des techniques d'attaque et de défense en matière de sécurité informatique.

N'hésitez pas à expérimenter avec différentes images et classes ciblées pour approfondir votre compréhension de l'attaque One Pixel et de ses implications.